# Veri Seti Yükleme

In [4]:
%cp /content/kaggle.json /root/.kaggle/

In [3]:
%mkdir /root/.kaggle/

In [5]:
!kaggle datasets download -d mengcius/cinic10

 99% 745M/754M [00:17<00:00, 44.3MB/s]
100% 754M/754M [00:17<00:00, 44.7MB/s]


In [6]:
import shutil
shutil.unpack_archive("cinic10.zip", "/content")

#Faster memory and Faster GPU


In [2]:


from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Your runtime has 38.0 gigabytes of available RAM

You are using a high-RAM runtime!
Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


# Birinci SORU

## Veri Setini Okuma

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, Conv2D, MaxPooling2D
from keras.callbacks import CSVLogger
#from livelossplot.keras import PlotLossesCallback
#import efficientnet.keras as efn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import os

In [2]:
# Hyperparams
IMAGE_SIZE = 224
IMAGE_WIDTH, IMAGE_HEIGHT = IMAGE_SIZE, IMAGE_SIZE
TEST_SIZE = 30

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)

In [3]:
from tqdm import tqdm

traindir = "/content/train"
testdir = "/content/test"
categories = ["airplane", "automobile", "bird","cat","deer","dog","frog","horse","ship","truck"]  

training_data = []
iterations = 0
test_data = []

for category in categories:
    path = os.path.join(traindir, category)  
    class_num = categories.index(category)  
    for img in tqdm(os.listdir(path)):
      iterations += 1
      if iterations > 500:
        break
      img_array = cv2.imread(os.path.join(path,img))
      new_array = cv2.resize(img_array, (IMAGE_SIZE, IMAGE_SIZE)) 
      training_data.append([new_array, class_num])
    iterations = 0


for category in categories:
    path = os.path.join(testdir, category)  
    class_num = categories.index(category)  
    for img in tqdm(os.listdir(path)):
      iterations += 1
      if iterations > 500:
        break
      img_array = cv2.imread(os.path.join(path,img))
      new_array = cv2.resize(img_array, (IMAGE_SIZE, IMAGE_SIZE)) 
      test_data.append([new_array, class_num])
    iterations = 0

print()
print(len(training_data))
print(len(test_data))

  4%|▍         | 368/9000 [00:00<00:03, 2682.04it/s]


5000
5000


In [4]:
import random
random.shuffle(training_data)

for sample in training_data[:10]:
  print(sample[1])

9
2
2
5
7
6
9
1
1
2


In [7]:
X_train, X_test = [], []
y_train, y_test = [], []

for features, label in training_data:
    X_train.append(features)
    y_train.append(label)

for features, label in test_data:
    X_test.append(features)
    y_test.append(label)

X_train = np.array(X_train).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
X_test = np.array(X_test).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)

In [7]:
print(X_train.shape)
print(X_test.shape)
print(len(y_train))
print(len(y_test))

(5000, 224, 224, 3)
(5000, 224, 224, 3)
5000
5000


In [9]:
X_train = np.array(X_train/255.0)
X_test = np.array(X_test/255.0)
y_train = np.array(y_train)
y_test = np.array(y_test)


In [10]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, len(categories))

In [11]:
print(y_train.shape)

(5000, 10)


## CNN model

In [11]:
#MODEL 1
model = Sequential()

model.add(Conv2D(32, (3,3), kernel_initializer='GlorotNormal', input_shape=input_shape, activation='relu'))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3,3), kernel_initializer='GlorotNormal', activation='relu'))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(len(categories), activation="softmax"))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

model.summary()
model.fit(X_train, y_train, batch_size=32,  epochs=10)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
dropout (Dropout)            (None, 222, 222, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 32)      9248      
_________________________________________________________________
dropout_1 (Dropout)          (None, 220, 220, 32)      0         
_________________________________________________________________
flatten (Flatten)            (None, 1548800)           0         
_________________________________________________________________
dense (Dense)                (None, 10)                15488010  
Total params: 15,498,154
Trainable params: 15,498,154
Non-trainable params: 0
____________________________________________

In [12]:
#MODEL 2  5x5 matrix
model = Sequential()

model.add(Conv2D(32, (5,5), kernel_initializer='GlorotNormal', input_shape=input_shape, activation='relu'))
model.add(Dropout(0.2))

model.add(Conv2D(32, (5,5), kernel_initializer='GlorotNormal', activation='relu'))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(len(categories), activation="softmax"))

In [13]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

model.summary()
model.fit(X_train, y_train, batch_size=32,  epochs=10)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 220, 220, 32)      2432      
_________________________________________________________________
dropout_2 (Dropout)          (None, 220, 220, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 216, 216, 32)      25632     
_________________________________________________________________
dropout_3 (Dropout)          (None, 216, 216, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1492992)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                14929930  
Total params: 14,957,994
Trainable params: 14,957,994
Non-trainable params: 0
__________________________________________

In [14]:

#MODEL 3 3x3 matrix, 0.7 Fropout
model = Sequential()

model.add(Conv2D(32, (3,3), kernel_initializer='GlorotNormal', input_shape=input_shape, activation='relu'))
model.add(Dropout(0.7))

model.add(Conv2D(32, (3,3), kernel_initializer='GlorotNormal', activation='relu'))
model.add(Dropout(0.7))

model.add(Flatten())
model.add(Dense(len(categories), activation="softmax"))

In [15]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

model.summary()
model.fit(X_train, y_train, batch_size=32,  epochs=10)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
dropout_4 (Dropout)          (None, 222, 222, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 220, 220, 32)      9248      
_________________________________________________________________
dropout_5 (Dropout)          (None, 220, 220, 32)      0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1548800)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                15488010  
Total params: 15,498,154
Trainable params: 15,498,154
Non-trainable params: 0
__________________________________________

In [11]:
#MODEL 4 3 KATMANLI
model = Sequential()

model.add(Conv2D(32, (3,3), kernel_initializer='GlorotNormal', input_shape=input_shape, activation='relu'))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3,3), kernel_initializer='GlorotNormal', activation='relu'))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3,3), kernel_initializer='GlorotNormal', activation='relu'))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(len(categories), activation="softmax"))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

model.summary()
model.fit(X_train, y_train, batch_size=32,  epochs=10)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
dropout (Dropout)            (None, 222, 222, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 32)      9248      
_________________________________________________________________
dropout_1 (Dropout)          (None, 220, 220, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 218, 218, 32)      9248      
_________________________________________________________________
dropout_2 (Dropout)          (None, 218, 218, 32)      0         
_________________________________________________________________
flatten (Flatten)            (None, 1520768)           0

In [12]:
#MODEL 5 3 KATMANLI
model = Sequential()

model.add(Conv2D(32, (5,5), kernel_initializer='GlorotNormal', input_shape=input_shape, activation='relu'))
model.add(Dropout(0.2))

model.add(Conv2D(32, (5,5), kernel_initializer='GlorotNormal', activation='relu'))
model.add(Dropout(0.2))

model.add(Conv2D(32, (5,5), kernel_initializer='GlorotNormal', activation='relu'))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(len(categories), activation="softmax"))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

model.summary()
model.fit(X_train, y_train, batch_size=32,  epochs=10)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 220, 220, 32)      2432      
_________________________________________________________________
dropout_3 (Dropout)          (None, 220, 220, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 216, 216, 32)      25632     
_________________________________________________________________
dropout_4 (Dropout)          (None, 216, 216, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 212, 212, 32)      25632     
_________________________________________________________________
dropout_5 (Dropout)          (None, 212, 212, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1438208)          

In [13]:
#MODEL 6 3 KATMANLI
model = Sequential()

model.add(Conv2D(32, (3,3), kernel_initializer='GlorotNormal', input_shape=input_shape, activation='relu'))
model.add(Dropout(0.7))

model.add(Conv2D(32, (3,3), kernel_initializer='GlorotNormal', activation='relu'))
model.add(Dropout(0.7))

model.add(Conv2D(32, (3,3), kernel_initializer='GlorotNormal', activation='relu'))
model.add(Dropout(0.7))

model.add(Flatten())
model.add(Dense(len(categories), activation="softmax"))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

model.summary()
model.fit(X_train, y_train, batch_size=32,  epochs=10)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
dropout_6 (Dropout)          (None, 222, 222, 32)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 220, 220, 32)      9248      
_________________________________________________________________
dropout_7 (Dropout)          (None, 220, 220, 32)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 218, 218, 32)      9248      
_________________________________________________________________
dropout_8 (Dropout)          (None, 218, 218, 32)      0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1520768)          

# İkinci SON

## VGG-16 1 Katmanlı Eğitim

In [ ]:
import keras
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import SGD
import tensorflow as tf
from keras import utils

In [ ]:
vgg_model = VGG16(weights="imagenet")  # imagenet ile eğitilecek

# remove the output layer
vgg_model = Model(inputs=vgg_model.inputs, outputs=vgg_model.layers[-2].output)

vgg_model.summary()

In [ ]:

fcc = Dense(1024, activation='relu', name='fcc')(vgg_model.layers[-2].output)
pred = Dense(3, activation='softmax', name='prediction')(fcc)
my_vgg16 = Model(vgg_model.input, pred)
for i in range(0, 21):
  my_vgg16.layers[i].trainable=False
my_vgg16.summary()

In [ ]:
_y_train = utils.to_categorical(y_train)
_y_test = utils.to_categorical(y_test)

In [ ]:

my_vgg16.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=["accuracy"])

my_vgg16.fit(X_train, _y_train, batch_size=64, epochs=10)

Epoch 1/10
24/24 [==============================] - 9s 277ms/step - loss: 5.4441 - accuracy: 0.3576
Epoch 2/10
24/24 [==============================] - 5s 227ms/step - loss: 0.6488 - accuracy: 0.5542
Epoch 3/10
24/24 [==============================] - 5s 229ms/step - loss: 0.5269 - accuracy: 0.5900
Epoch 4/10
24/24 [==============================] - 5s 229ms/step - loss: 0.5402 - accuracy: 0.6023
Epoch 5/10
24/24 [==============================] - 6s 231ms/step - loss: 0.4571 - accuracy: 0.6883
Epoch 6/10
24/24 [==============================] - 6s 231ms/step - loss: 0.4199 - accuracy: 0.7111
Epoch 7/10
24/24 [==============================] - 6s 232ms/step - loss: 0.4300 - accuracy: 0.7282
Epoch 8/10
24/24 [==============================] - 6s 232ms/step - loss: 0.4332 - accuracy: 0.7197
Epoch 9/10
24/24 [==============================] - 6s 233ms/step - loss: 0.4394 - accuracy: 0.7188
Epoch 10/10
24/24 [==============================] - 6s 233ms/step - loss: 0.3652 - accuracy: 0.7694

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

y_pred = my_model.predict(X_test)
y_pred = np.argmax(y_pred, axis = 1)
_y_test = np.argmax(_y_test, axis = 1)

print(confusion_matrix(_y_test, y_pred))
print(classification_report(_y_test, y_pred))

[[297  48 155]
 [ 49 337 114]
 [ 25  12 463]]
              precision    recall  f1-score   support

           0       0.80      0.59      0.68       500
           1       0.85      0.67      0.75       500
           2       0.63      0.93      0.75       500

    accuracy                           0.73      1500
   macro avg       0.76      0.73      0.73      1500
weighted avg       0.76      0.73      0.73      1500



## VGG-16 4 Katmanlı Eğitim

In [ ]:
import keras
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import SGD
import tensorflow as tf
from keras import utils

In [ ]:
vgg_model = VGG16(weights="imagenet")  # imagenet ile eğitilecek

# remove the output layer
vgg_model = Model(inputs=vgg_model.inputs, outputs=vgg_model.layers[-2].output)

vgg_model.summary()

In [ ]:
fcc = Dense(1024, activation='relu', name='fcc')(vgg_model.layers[-2].output)
pred = Dense(len(categories), activation='softmax', name='prediction')(fcc)
my_vgg16 = Model(vgg_model.input, pred)
for i in range(0, 19):
  my_vgg16.layers[i].trainable=False
my_vgg16.summary()

In [ ]:
_y_train = utils.to_categorical(y_train)
_y_test = utils.to_categorical(y_test)

In [ ]:

my_vgg16.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=["accuracy"])

my_vgg16.fit(X_train, _y_train, batch_size=64, epochs=10)

Epoch 1/10
24/24 [==============================] - 7s 249ms/step - loss: 7.5399 - accuracy: 0.3814
Epoch 2/10
24/24 [==============================] - 6s 248ms/step - loss: 0.3398 - accuracy: 0.7908
Epoch 3/10
24/24 [==============================] - 6s 250ms/step - loss: 0.2350 - accuracy: 0.8565
Epoch 4/10
24/24 [==============================] - 6s 251ms/step - loss: 0.1656 - accuracy: 0.9028
Epoch 5/10
24/24 [==============================] - 6s 251ms/step - loss: 0.1027 - accuracy: 0.9464
Epoch 6/10
24/24 [==============================] - 6s 251ms/step - loss: 0.0748 - accuracy: 0.9684
Epoch 7/10
24/24 [==============================] - 6s 252ms/step - loss: 0.0648 - accuracy: 0.9596
Epoch 8/10
24/24 [==============================] - 6s 252ms/step - loss: 0.0566 - accuracy: 0.9755
Epoch 9/10
24/24 [==============================] - 6s 252ms/step - loss: 0.0744 - accuracy: 0.9508
Epoch 10/10
24/24 [==============================] - 6s 254ms/step - loss: 0.0214 - accuracy: 0.9903

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

y_pred = my_model.predict(X_test)
y_pred = np.argmax(y_pred, axis = 1)
_y_test = np.argmax(_y_test, axis = 1)

print(confusion_matrix(_y_test,y_pred))
print(classification_report(_y_test,y_pred))

## Resnet-50 1 Katmanlı Eğitim

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.models import Sequential, Model
from keras.layers import Dense
from keras import utils

In [ ]:
resnet_model = ResNet50(weights='imagenet')
resnet_model.summary()

In [ ]:
# add 1 fully-connected and 1 prediction
fcc = Dense(1024, name='fcc')(resnet_model.layers[-2].output)
pred = Dense(len(categories), activation='softmax', name='prediction')(fcc)
my_resnet50 = Model(resnet_model.input, pred)
for i in range(0, 175):
  my_resnet50.layers[i].trainable=False
my_resnet50.summary()

In [ ]:
_y_train = utils.to_categorical(y_train)
_y_test = utils.to_categorical(y_test)

In [ ]:
my_resnet50.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=["accuracy"])

my_resnet50.fit(X_train, _y_train, batch_size=32, epochs=10)

Epoch 1/10
47/47 [==============================] - 14s 99ms/step - loss: 1.0695 - accuracy: 0.3350
Epoch 2/10
47/47 [==============================] - 4s 92ms/step - loss: 0.6657 - accuracy: 0.4212
Epoch 3/10
47/47 [==============================] - 4s 92ms/step - loss: 0.5923 - accuracy: 0.4884
Epoch 4/10
47/47 [==============================] - 4s 92ms/step - loss: 0.6522 - accuracy: 0.4283
Epoch 5/10
47/47 [==============================] - 4s 93ms/step - loss: 0.5850 - accuracy: 0.4971
Epoch 6/10
47/47 [==============================] - 4s 93ms/step - loss: 0.5656 - accuracy: 0.5273
Epoch 7/10
47/47 [==============================] - 4s 92ms/step - loss: 0.5696 - accuracy: 0.5576
Epoch 8/10
47/47 [==============================] - 4s 93ms/step - loss: 0.5653 - accuracy: 0.5228
Epoch 9/10
47/47 [==============================] - 4s 92ms/step - loss: 0.5780 - accuracy: 0.5351
Epoch 10/10
47/47 [==============================] - 4s 93ms/step - loss: 0.5646 - accuracy: 0.5616


In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

y_pred = my_resnet50.predict(X_test)
y_pred = np.argmax(y_pred, axis = 1)
_y_test = np.argmax(_y_test, axis = 1)

print(confusion_matrix(_y_test,y_pred))
print(classification_report(_y_test,y_pred))

[[ 96 214 190]
 [  6 403  91]
 [  4 115 381]]
              precision    recall  f1-score   support

           0       0.91      0.19      0.32       500
           1       0.55      0.81      0.65       500
           2       0.58      0.76      0.66       500

    accuracy                           0.59      1500
   macro avg       0.68      0.59      0.54      1500
weighted avg       0.68      0.59      0.54      1500



## Resnet-50 4 Katmanlı Eğitim

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.models import Sequential, Model
from keras.layers import Dense
from keras import utils

In [ ]:
resnet_model = ResNet50(weights='imagenet')
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
# add 1 fully-connected and 1 prediction
fcc = Dense(1024, name='fcc')(resnet_model.layers[-2].output)
pred = Dense(len(categories), activation='softmax', name='prediction')(fcc)
my_resnet50 = Model(resnet_model.input, pred)
for i in range(0, 173):  # only last 4 will be trainable
  my_resnet50.layers[i].trainable=False
my_resnet50.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [ ]:
_y_train = utils.to_categorical(y_train)
_y_test = utils.to_categorical(y_test)

In [ ]:
my_resnet50.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=["accuracy"])

my_resnet50.fit(X_train, _y_train, batch_size=32, epochs=10)

Epoch 1/10
47/47 [==============================] - 9s 98ms/step - loss: 1.0242 - accuracy: 0.3292
Epoch 2/10
47/47 [==============================] - 4s 91ms/step - loss: 0.6524 - accuracy: 0.4133
Epoch 3/10
47/47 [==============================] - 4s 92ms/step - loss: 0.6171 - accuracy: 0.4672
Epoch 4/10
47/47 [==============================] - 4s 92ms/step - loss: 0.6079 - accuracy: 0.4473
Epoch 5/10
47/47 [==============================] - 4s 92ms/step - loss: 0.5941 - accuracy: 0.4757
Epoch 6/10
47/47 [==============================] - 4s 92ms/step - loss: 0.6042 - accuracy: 0.4845
Epoch 7/10
47/47 [==============================] - 4s 92ms/step - loss: 0.5828 - accuracy: 0.5268
Epoch 8/10
47/47 [==============================] - 4s 92ms/step - loss: 0.5640 - accuracy: 0.5432
Epoch 9/10
47/47 [==============================] - 4s 93ms/step - loss: 0.5626 - accuracy: 0.5403
Epoch 10/10
47/47 [==============================] - 4s 92ms/step - loss: 0.5314 - accuracy: 0.5903


In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

y_pred = my_resnet50.predict(X_test)
y_pred = np.argmax(y_pred, axis = 1)
_y_test = np.argmax(_y_test, axis = 1)

print(confusion_matrix(_y_test,y_pred))
print(classification_report(_y_test,y_pred))

[[288 188  24]
 [ 77 418   5]
 [208 162 130]]
              precision    recall  f1-score   support

           0       0.50      0.58      0.54       500
           1       0.54      0.84      0.66       500
           2       0.82      0.26      0.39       500

    accuracy                           0.56      1500
   macro avg       0.62      0.56      0.53      1500
weighted avg       0.62      0.56      0.53      1500



# VGG H5 Model Weight Load

In [ ]:
VGG16_H_PATH = ""



In [ ]:
#Load Weights
VGG16_model = model.load_weights(VGG16_H_PATH)

VGG16_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=["accuracy"])

VGG16_model.fit(X_train, _y_train, batch_size=32, epochs=10)

# Re-evaluate the model
loss, acc = VGG16_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

#ResNet50 Model Weight Load

In [ ]:
VGG16_H_PATH = ""


In [ ]:
#Load Weights
ResNet_model = model.load_weights(VGG16_H_PATH)

ResNet_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=["accuracy"])

ResNet_model.fit(X_train, _y_train, batch_size=32, epochs=10)

# Re-evaluate the model
loss, acc = ResNet_model.evaluate(_y_test, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))